In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [19]:
# ---- STEP 1: Load pyranometer data ----
# Update the path to your CSV file
pyrano = pd.read_csv("Sunshine Pyranometer/25.05.02_CELLS_SPN1_global, diffuse.csv")

# Keep only rows between 10:10 and 10:25 
pyrano_filtered = pyrano[(pyrano['timestamp.1'] >= '10:10:00') & (pyrano['timestamp.1'] <= '10:25:00')].copy()


In [20]:
# ---- STEP 2: Load CUBE6 data and compute average ----
# Load CUBE6 data
cube6 = pd.read_csv("New Cube Sensors/25.05.02_CUBE 6.csv")

# Define relevant column names
time_col = ' HH:MM:SS'
sw_col = ' SOUTH SW RADIANCE (W/m^2/sr)'

# Filter by time window (string comparison is fine for this format)
cube6_filtered = cube6[(cube6[time_col] >= '10:10:00') & (cube6[time_col] <= '10:25:00')].copy()

# Convert HH:MM:SS to datetime.time so we can group by minute
cube6_filtered.loc[:, 'minute'] = pd.to_datetime(cube6_filtered[time_col], format="%H:%M:%S").dt.strftime('%H:%M')

# Compute average SOUTH SW RADIANCE per minute
minute_avg = cube6_filtered.groupby('minute')[sw_col].mean()

# Compute overall mean of the minute-averaged values
I_transmitted_CUBE6 = minute_avg.mean()


# Print results
#print(minute_avg)
#print(f"\nFinal average SOUTH SW RADIANCE over 10:10–10:25 (minute-averaged): {I_transmitted_CUBE6:.2f} W/m²/sr")

In [24]:
# ---- STEP 4: Use fixed sun position at 10:15 CEST (Fribourg) ----
# Sun position on May 2, 2025 at 10:15 CEST:
alpha_deg = 36.2       # solar altitude
azimuth_deg = 122.6    # solar azimuth
window_azimuth_deg = 180  # south-facing window

# Convert degrees to radians
alpha = np.radians(alpha_deg)
azimuth = np.radians(azimuth_deg)
window_azimuth = np.radians(window_azimuth_deg)

# ---- STEP 5: Compute solar components on the window ----

# Use original column names for GHI and DHI
GHI_col = 'CELLS/Outside/SPN1/Global'
DHI_col = 'CELLS/Outside/SPN1/Diffuse'

# Estimate DNI from horizontal components
pyrano_filtered['DNI'] = (pyrano_filtered[GHI_col] - pyrano_filtered[DHI_col]) / np.sin(alpha)
pyrano_filtered['DNI'] = pyrano_filtered['DNI'].clip(lower=0)

# Compute incidence angle cosine (corrected formula for vertical window)
cos_theta_i = np.cos(alpha) * np.cos(azimuth - window_azimuth)
cos_theta_i = np.clip(cos_theta_i, 0, 1)


# Beam and diffuse irradiance on the vertical window
pyrano_filtered['I_beam_vert'] = pyrano_filtered['DNI'] * cos_theta_i
pyrano_filtered['I_diffuse_vert'] = 0.5 * pyrano_filtered[DHI_col]
pyrano_filtered['I_incident'] = pyrano_filtered['I_beam_vert'] + pyrano_filtered['I_diffuse_vert']



In [25]:
# ---- STEP 6: Show diffuse contribution ----
diffuse_ratio = pyrano_filtered['I_diffuse_vert'].mean() / pyrano_filtered['I_incident'].mean()
print(f"Diffuse fraction of incident irradiance: {diffuse_ratio:.2%}")



# ---- STEP 8: Estimate g-value ----
I_incident_avg = pyrano_filtered['I_incident'].mean()
g_value = I_transmitted_CUBE6 / I_incident_avg

print(f"Average transmitted irradiance (CUBE6): {I_transmitted_CUBE6:.2f} W/m²")
print(f"Average incident irradiance on window: {I_incident_avg:.2f} W/m²")
print(f"Estimated g-value: {g_value:.3f}")

Diffuse fraction of incident irradiance: 13.62%
Average transmitted irradiance (CUBE6): 8.20 W/m²
Average incident irradiance on window: 425.93 W/m²
Estimated g-value: 0.019
